In [1]:
from moccasin import setup_notebook

setup_notebook()

In [2]:
from moccasin.config import get_active_network, Network

active_network: Network = get_active_network()
print(active_network.name)

eth-forked


In [3]:
import boa
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network, Network

STARTTING_ETH_BALANCE = int(1000e18)
STARTTING_WETH_BALANCE = int(1e18)
STARTTING_USDC_BALANCE = int(100e6)  # usdc 6 decimals!


def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTTING_ETH_BALANCE)


def _add_token_balance(usdc: ABIContract, weth: ABIContract):
    our_address = boa.env.eoa

    print(f"Starting balance of WETH: {weth.balanceOf(our_address)}")
    weth.deposit(value=STARTTING_WETH_BALANCE)
    print(f"Ending balance of WETH: {weth.balanceOf(our_address)}")

    print(f"Starting balance of USDC: {usdc.balanceOf(our_address)}")
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTTING_USDC_BALANCE)
    usdc.mint(our_address, STARTTING_USDC_BALANCE)
    print(f"Ending balance of USDC: {usdc.balanceOf(our_address)}")


def setup_script() -> Tuple[ABIContract, ABIContract, ABIContract, ABIContract]:
    print("Starting setup script...")
    active_network: Network = get_active_network()

    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth)


def moccasin_main():
    setup_script()

moccasin_main()

Starting setup script...
Starting balance of WETH: 0
Ending balance of WETH: 1000000000000000000
Starting balance of USDC: 0
Ending balance of USDC: 100000000


In [4]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [5]:
weth.balanceOf(boa.env.eoa)
usdc.balanceOf(boa.env.eoa)

100000000

In [8]:
from moccasin.config import get_or_initialize_config, Config

config: Config = get_or_initialize_config()
config.reload()
active_network = config.get_active_network()
aavev3_pool_address_provider = active_network.manifest_named("aavev3_pool_address_provider")
pool_address = aavev3_pool_address_provider.getPool()
print(pool_address)

config.reload()
active_network = config.get_active_network()
pool_contract = active_network.manifest_named("pool", address=pool_address)

0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [10]:
REFERAL_CODE = 0

def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {amount} {token.name()} into Aave contract pool {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)


(
    totalCollateralBase,
    totalDebtBase,
    availableBorrowsBase,
    currentLiquidationThreshold,
    ltv,
    healthFactor,
) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"""User account data:
        totalCollateralBase: {totalCollateralBase}
        totalDebtBase: {totalDebtBase}
        availableBorrowsBase: {availableBorrowsBase}
        currentLiquidationThreshold: {currentLiquidationThreshold}
        ltv: {ltv}
        healthFactor: {healthFactor}
          """)

Depositing 100000000 USD Coin into Aave contract pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
Depositing 1000000000000000000 Wrapped Ether into Aave contract pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
User account data:
        totalCollateralBase: 322869956374
        totalDebtBase: 0
        availableBorrowsBase: 259329148960
        currentLiquidationThreshold: 8284
        ltv: 8032
        healthFactor: 115792089237316195423570985008687907853269984665640564039457584007913129639935
          
